In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt langgraph_sdk langgraph-checkpoint-sqlite langsmith langchain-community google-search-results wikipedia amadeus langgraph requests

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import operator
import sqlite3
import json
import os
import uuid
from typing import List, Annotated, TypedDict
from datetime import datetime
from langchain_core.messages import AnyMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.types import Send
from langgraph.checkpoint.sqlite import SqliteSaver
from amadeus import Client, ResponseError


# 1. Amadeus API Setup

try:
    amadeus_client = Client(
        client_id=os.environ.get('AMADEUS_CLIENT_ID'),
        client_secret=os.environ.get('AMADEUS_CLIENT_SECRET')
    )
except Exception:
    amadeus_client = None


# 2. Helping Functions

def convert_city_to_airport_code(city_name):
    """Convert city name like 'New York' to airport code like 'NYC'"""
    if amadeus_client is None:
        return city_name
    
    try:
        api_response = amadeus_client.reference_data.locations.get(
            keyword=city_name, 
            subType='CITY'
        )
        
        if api_response.data:
            airport_code = api_response.data[0]['iataCode']
            print(f"   Converted '{city_name}' -> '{airport_code}'")
            return airport_code
    except Exception:
        pass
    
    return city_name


def check_if_dates_are_valid(arrival_date_string, return_date_string):
    """Check if travel dates are valid (in future and return after arrival)"""
    try:
        today = datetime.now().date()
        arrival_date = datetime.strptime(arrival_date_string, "%Y-%m-%d").date()
        return_date = datetime.strptime(return_date_string, "%Y-%m-%d").date()
        
        if arrival_date < today:
            error_message = "Arrival date cannot be in the past. Please select a future date."
            return False, error_message
        
        if return_date < today:
            error_message = "Return date cannot be in the past. Please select a future date."
            return False, error_message
        
        if return_date <= arrival_date:
            error_message = "Return date must be after arrival date."
            return False, error_message
        
        return True, "Dates are valid"
    except ValueError:
        error_message = "Invalid date format. Please use YYYY-MM-DD format."
        return False, error_message


def convert_json_string_to_list(json_string):
    """Convert JSON string to Python list"""
    try:
        data = json.loads(json_string)
        if isinstance(data, list):
            return data
        else:
            return []
    except Exception:
        return []


def replace_old_value_with_new(old_value, new_value):
    """Simple function to replace values in state"""
    return new_value


# 3. Defining the State Structure

class TravelAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    origin: Annotated[str, replace_old_value_with_new]
    destination: Annotated[str, replace_old_value_with_new]
    currency: Annotated[str, replace_old_value_with_new]
    budget: Annotated[str, replace_old_value_with_new]
    num_people: Annotated[str, replace_old_value_with_new]
    arrival_date: Annotated[str, replace_old_value_with_new]
    stay_duration: Annotated[str, replace_old_value_with_new]
    return_date: Annotated[str, replace_old_value_with_new]
    flight_options: Annotated[List[dict], operator.add]
    hotel_options: Annotated[List[dict], operator.add]
    date_error: Annotated[str, replace_old_value_with_new]



# 4. Defining Core Tools


@tool
def search_flight(origin, destination, date, currency="USD"):
    """Search for flights between two cities on a specific date"""
    
    if amadeus_client is None:
        empty_result = []
        return json.dumps(empty_result)
    
    print(f"\n   [AMADEUS] Searching Flights: {origin} -> {destination} on {date}")
    
    try:
        origin_airport_code = convert_city_to_airport_code(origin)
        destination_airport_code = convert_city_to_airport_code(destination)
        
        api_response = amadeus_client.shopping.flight_offers_search.get(
            originLocationCode=origin_airport_code,
            destinationLocationCode=destination_airport_code,
            departureDate=date,
            adults=1,
            currencyCode=currency,
            max=3
        )
        
        flight_results = []
        
        for flight in api_response.data[:3]:
            flight_price = float(flight['price']['total'])
            flight_currency = flight['price']['currency']
            flight_segments = flight['itineraries'][0]['segments']
            airline_code = flight_segments[0]['carrierCode']
            
            departure_time_full = flight_segments[0]['departure']['at']
            arrival_time_full = flight_segments[-1]['arrival']['at']
            
            if 'T' in departure_time_full:
                departure_time = departure_time_full.split('T')[1][:5]
            else:
                departure_time = departure_time_full
            
            if 'T' in arrival_time_full:
                arrival_time = arrival_time_full.split('T')[1][:5]
            else:
                arrival_time = arrival_time_full
            
            try:
                date_object = datetime.strptime(date, "%Y-%m-%d")
                formatted_date = date_object.strftime("%y%m%d")
            except:
                formatted_date = date.replace("-", "")[2:]
            
            skyscanner_url = (
                f"https://www.skyscanner.com/transport/flights/"
                f"{origin_airport_code.lower()}/{destination_airport_code.lower()}/{formatted_date}/"
                f"?adults=1&cabinclass=economy&rtn=0"
            )
            
            flight_info = {
                "type": "flight",
                "airline": airline_code,
                "route": f"{origin_airport_code} -> {destination_airport_code}",
                "origin_code": origin_airport_code,
                "destination_code": destination_airport_code,
                "price": flight_price,
                "currency": flight_currency,
                "departure": departure_time,
                "arrival": arrival_time,
                "date": date,
                "link": skyscanner_url
            }
            
            flight_results.append(flight_info)
        
        print(f"   Found {len(flight_results)} flights")
        return json.dumps(flight_results)
    
    except ResponseError:
        empty_result = []
        return json.dumps(empty_result)
    except Exception:
        empty_result = []
        return json.dumps(empty_result)


@tool
def search_hotel(location, checkin_date, checkout_date, currency="USD"):
    """Search for hotels in a city for specific dates"""
    
    if amadeus_client is None:
        empty_result = []
        return json.dumps(empty_result)
    
    print(f"\n   [AMADEUS] Searching Hotels in: {location}")
    
    try:
        city_airport_code = convert_city_to_airport_code(location)
        
        hotels_api_response = amadeus_client.reference_data.locations.hotels.by_city.get(
            cityCode=city_airport_code
        )
        
        if not hotels_api_response.data:
            print(f"   No hotels found in {location}")
            location_url_encoded = location.replace(' ', '+')
            fallback_url = f"https://www.booking.com/searchresults.html?ss={location_url_encoded}&checkin={checkin_date}&checkout={checkout_date}"
            
            fallback_hotel = {
                "type": "hotel",
                "name": f"Hotels in {location}",
                "price": None,
                "currency": currency,
                "rating": 0,
                "rating_word": "",
                "stars": 0,
                "address": location,
                "distance_to_center": "Various",
                "link": fallback_url
            }
            
            return json.dumps([fallback_hotel])
        
        hotel_results = []
        hotel_id_list = []
        
        for hotel in hotels_api_response.data[:5]:
            hotel_id = hotel['hotelId']
            hotel_id_list.append(hotel_id)
        
        hotel_ids_joined = ','.join(hotel_id_list)
        
        try:
            offers_api_response = amadeus_client.shopping.hotel_offers_search.get(
                hotelIds=hotel_ids_joined,
                checkInDate=checkin_date,
                checkOutDate=checkout_date,
                adults=1,
                currency=currency
            )
            
            for hotel_offer in offers_api_response.data:
                hotel_name = hotel_offer['hotel']['name']
                hotel_id = hotel_offer['hotel']['hotelId']
                
                hotel_price = None
                hotel_currency = currency
                
                if hotel_offer.get('offers') and len(hotel_offer['offers']) > 0:
                    hotel_price = float(hotel_offer['offers'][0]['price']['total'])
                    hotel_currency = hotel_offer['offers'][0]['price']['currency']
                
                hotel_name_url_encoded = hotel_name.replace(" ", "+")
                location_url_encoded = location.replace(' ', '+')
                booking_url = f"https://www.booking.com/searchresults.html?ss={hotel_name_url_encoded}+{location_url_encoded}&checkin={checkin_date}&checkout={checkout_date}&selected_currency={currency}"
                
                hotel_info = {
                    "type": "hotel",
                    "name": hotel_name,
                    "price": hotel_price,
                    "currency": hotel_currency,
                    "rating": 0,
                    "rating_word": "",
                    "stars": 0,
                    "address": location,
                    "distance_to_center": "N/A",
                    "link": booking_url
                }
                
                hotel_results.append(hotel_info)
        
        except ResponseError:
            for hotel in hotels_api_response.data[:5]:
                hotel_name = hotel.get('name', 'Unknown Hotel')
                hotel_name_url_encoded = hotel_name.replace(" ", "+")
                location_url_encoded = location.replace(' ', '+')
                booking_url = f"https://www.booking.com/searchresults.html?ss={hotel_name_url_encoded}+{location_url_encoded}&checkin={checkin_date}&checkout={checkout_date}&selected_currency={currency}"
                
                hotel_info = {
                    "type": "hotel",
                    "name": hotel_name,
                    "price": None,
                    "currency": currency,
                    "rating": 0,
                    "rating_word": "",
                    "stars": 0,
                    "address": location,
                    "distance_to_center": "N/A",
                    "link": booking_url
                }
                
                hotel_results.append(hotel_info)
        
        print(f"   Found {len(hotel_results)} hotels")
        return json.dumps(hotel_results)
    
    except ResponseError:
        empty_result = []
        return json.dumps(empty_result)
    except Exception:
        empty_result = []
        return json.dumps(empty_result)



# 6. Defining The Travel Sub-Graph 

def travel_agent_node(state):
    """This function searches for both outbound and return flights"""
    print("\nTravel Agent: Searching flights...")
    
    user_currency = state.get('currency', 'USD')
    
    outbound_flight_result = search_flight.invoke({
        "origin": state['origin'],
        "destination": state['destination'],
        "date": state['arrival_date'],
        "currency": user_currency
    })
    
    return_flight_result = search_flight.invoke({
        "origin": state['destination'],
        "destination": state['origin'],
        "date": state['return_date'],
        "currency": user_currency
    })
    
    outbound_flights = convert_json_string_to_list(outbound_flight_result)
    return_flights = convert_json_string_to_list(return_flight_result)
    
    all_flights = outbound_flights + return_flights
    
    print(f"   Total flights found: {len(all_flights)}")
    
    return {"flight_options": all_flights}


travel_graph_builder = StateGraph(TravelAgentState)
travel_graph_builder.add_node("travel_agent", travel_agent_node)
travel_graph_builder.add_edge(START, "travel_agent")
travel_graph_builder.add_edge("travel_agent", END)
travel_graph = travel_graph_builder.compile()


# 7. Defining The Hotel Sub-Graph

def hotel_agent_node(state):
    """This function searches for hotels in the destination city"""
    print("\nHotel Agent: Searching hotels...")
    
    user_currency = state.get('currency', 'USD')
    
    hotel_search_result = search_hotel.invoke({
        "location": state['destination'],
        "checkin_date": state['arrival_date'],
        "checkout_date": state['return_date'],
        "currency": user_currency
    })
    
    hotels = convert_json_string_to_list(hotel_search_result)
    
    print(f"   Total hotels found: {len(hotels)}")
    
    return {"hotel_options": hotels}


hotel_graph_builder = StateGraph(TravelAgentState)
hotel_graph_builder.add_node("hotel_agent", hotel_agent_node)
hotel_graph_builder.add_edge(START, "hotel_agent")
hotel_graph_builder.add_edge("hotel_agent", END)
hotel_graph = hotel_graph_builder.compile()

# 8. Building the Main Node Functions

def intake_node(state):
    """Initialize empty lists for flight and hotel options"""
    return {
        "flight_options": [],
        "hotel_options": []
    }


def planner_node(state):
    """Validate dates and plan the trip"""
    print(f"\nPlanning trip: {state['origin']} -> {state['destination']}")
    print(f"   Dates: {state['arrival_date']} to {state['return_date']}")
    print(f"   Budget: {state.get('budget', 'N/A')} {state.get('currency', 'USD')}")
    
    dates_are_valid, validation_message = check_if_dates_are_valid(
        state['arrival_date'], 
        state['return_date']
    )
    
    if not dates_are_valid:
        return {"date_error": validation_message}
    
    return {}


def decide_next_step_after_planning(state):
    """Decide whether to show error or continue searching"""
    if state.get('date_error'):
        return "present_plan"
    return "specialists"


def route_to_both_agents(state):
    """Send state to both travel agent and hotel agent in parallel"""
    return [
        Send("travel_agent", state),
        Send("accommodation_agent", state)
    ]


def present_plan_node(state):
    """Format and display the final trip plan"""
    
    if state.get('date_error'):
        error_message = state.get('date_error')
        error_text = "# Trip Planning Error\n\n"
        error_text += f"**Error:** {error_message}\n\n"
        error_text += "Please try again with valid dates:\n"
        error_text += "- Arrival and return dates must be in the future\n"
        error_text += "- Return date must be after arrival date\n"
        error_text += "- Use YYYY-MM-DD format (e.g., 2025-06-15)\n"
        
        return {"messages": [AIMessage(content=error_text)]}
    
    flights = state.get("flight_options", [])
    hotels = state.get("hotel_options", [])
    user_currency = state.get('currency', 'USD')
    budget = float(state.get('budget', 0))
    
    arrival_date_object = datetime.strptime(state['return_date'], "%Y-%m-%d")
    return_date_object = datetime.strptime(state['arrival_date'], "%Y-%m-%d")
    num_nights = (arrival_date_object - return_date_object).days
    
    output_text = f"# Your Trip Plan: {state['origin']} <-> {state['destination']}\n\n"
    output_text += f"**Dates:** {state['arrival_date']} to {state['return_date']} ({num_nights} nights)\n"
    output_text += f"**Budget:** {budget:.2f} {user_currency}\n"
    output_text += f"**Travelers:** {state.get('num_people', '1')}\n\n"
    output_text += "---\n\n"
    
    output_text += "## Flight Options\n\n"
    
    if flights:
        outbound_flights = []
        return_flights = []
        
        for flight in flights:
            origin_code = flight.get('origin_code', '')
            destination_code = flight.get('destination_code', '')
            
            user_origin_code = convert_city_to_airport_code(state['origin'])
            user_destination_code = convert_city_to_airport_code(state['destination'])
            
            if origin_code == user_origin_code and destination_code == user_destination_code:
                outbound_flights.append(flight)
            elif origin_code == user_destination_code and destination_code == user_origin_code:
                return_flights.append(flight)
        
        if outbound_flights:
            output_text += "### Outbound Flights\n\n"
            
            for index, flight in enumerate(outbound_flights, 1):
                flight_price = flight.get('price', 0)
                flight_currency = flight.get('currency', user_currency)
                airline = flight.get('airline', 'Unknown')
                route = flight.get('route', '')
                departure = flight.get('departure', 'N/A')
                arrival = flight.get('arrival', 'N/A')
                link = flight.get('link', '#')
                
                output_text += f"**{index}. {airline}** - {route}\n"
                output_text += f"   Price: {flight_price:.2f} {flight_currency}"
                
                if flight_currency != user_currency:
                    output_text += " (Currency mismatch)\n"
                else:
                    output_text += "\n"
                
                output_text += f"   Departs {departure} | Arrives {arrival}\n"
                output_text += f"   [Search on Skyscanner]({link})\n\n"
        
        if return_flights:
            output_text += "### Return Flights\n\n"
            
            for index, flight in enumerate(return_flights, 1):
                flight_price = flight.get('price', 0)
                flight_currency = flight.get('currency', user_currency)
                airline = flight.get('airline', 'Unknown')
                route = flight.get('route', '')
                departure = flight.get('departure', 'N/A')
                arrival = flight.get('arrival', 'N/A')
                link = flight.get('link', '#')
                
                output_text += f"**{index}. {airline}** - {route}\n"
                output_text += f"   Price: {flight_price:.2f} {flight_currency}"
                
                if flight_currency != user_currency:
                    output_text += " (Currency mismatch)\n"
                else:
                    output_text += "\n"
                
                output_text += f"   Departs {departure} | Arrives {arrival}\n"
                output_text += f"   [Search on Skyscanner]({link})\n\n"
        
        if outbound_flights and return_flights:
            cheapest_outbound = None
            lowest_outbound_price = float('inf')
            
            for flight in outbound_flights:
                flight_price = flight.get('price', float('inf'))
                if flight_price < lowest_outbound_price:
                    lowest_outbound_price = flight_price
                    cheapest_outbound = flight
            
            cheapest_return = None
            lowest_return_price = float('inf')
            
            for flight in return_flights:
                flight_price = flight.get('price', float('inf'))
                if flight_price < lowest_return_price:
                    lowest_return_price = flight_price
                    cheapest_return = flight
            
            total_flight_cost = cheapest_outbound.get('price', 0) + cheapest_return.get('price', 0)
            output_text += f"**Best Flight Deal:** {total_flight_cost:.2f} {user_currency} (outbound + return)\n\n"
    else:
        output_text += "No flights found. Check API configuration or try different dates.\n\n"
    
    output_text += "---\n\n"
    output_text += "## Hotel Options\n\n"
    
    if hotels:
        for index, hotel in enumerate(hotels, 1):
            hotel_name = hotel.get('name', 'Unknown Hotel')
            hotel_price = hotel.get('price')
            hotel_currency = hotel.get('currency', user_currency)
            rating = hotel.get('rating', 0)
            rating_word = hotel.get('rating_word', '')
            stars = hotel.get('stars', 0)
            distance = hotel.get('distance_to_center', 'N/A')
            link = hotel.get('link', '#')
            
            output_text += f"**{index}. {hotel_name}**"
            
            if stars > 0:
                output_text += f" {stars}-star\n"
            else:
                output_text += "\n"
            
            if hotel_price:
                total_hotel_cost = hotel_price * num_nights
                output_text += f"   Price: {hotel_price:.2f} {hotel_currency}/night x {num_nights} nights = {total_hotel_cost:.2f} {hotel_currency}\n"
            else:
                output_text += "   Price: See website for pricing\n"
            
            if rating > 0:
                output_text += f"   Rating: {rating}/10 ({rating_word})\n"
            
            output_text += f"   Location: {distance} from center\n"
            output_text += f"   [View & Book on Booking.com]({link})\n\n"
        
        if flights and outbound_flights and return_flights:
            hotels_with_prices = []
            
            for hotel in hotels:
                if hotel.get('price'):
                    hotels_with_prices.append(hotel)
            
            if hotels_with_prices:
                cheapest_hotel = None
                lowest_hotel_price = float('inf')
                
                for hotel in hotels_with_prices:
                    hotel_price = hotel.get('price', float('inf'))
                    if hotel_price < lowest_hotel_price:
                        lowest_hotel_price = hotel_price
                        cheapest_hotel = hotel
                
                cheapest_hotel_price = cheapest_hotel.get('price', 0)
                total_hotel_cost = cheapest_hotel_price * num_nights
                total_cost = total_flight_cost + total_hotel_cost
                
                output_text += "**Budget Analysis:**\n"
                output_text += f"   - Flights (cheapest): {total_flight_cost:.2f} {user_currency}\n"
                output_text += f"   - Hotel (cheapest): {total_hotel_cost:.2f} {user_currency}\n"
                output_text += f"   - **Total:** {total_cost:.2f} {user_currency}\n"
                
                if total_cost <= budget:
                    remaining_budget = budget - total_cost
                    output_text += f"   Within budget! ({remaining_budget:.2f} {user_currency} remaining)\n\n"
                else:
                    over_budget = total_cost - budget
                    output_text += f"   Over budget by: {over_budget:.2f} {user_currency}\n\n"
    else:
        output_text += "No hotels found. Check API configuration or try different destination.\n\n"
    
    output_text += "---\n\n"
    output_text += "### Important Notes:\n"
    output_text += "- **Data Source:** Flights and Hotels both from Amadeus API\n"
    output_text += "- **Pricing:** API data may not reflect live prices. Click links for current rates.\n"
    output_text += "- **Booking:** All links open search pages where you can view details and book.\n"
    output_text += "- **International:** Amadeus API supports worldwide destinations including India, Africa, Asia, etc.\n"
    
    return {"messages": [AIMessage(content=output_text)]}


# 9. Building the Main Graph

database_connection = sqlite3.connect("travel_planner.db", check_same_thread=False)
memory_saver = SqliteSaver(conn=database_connection)

workflow_builder = StateGraph(TravelAgentState)

workflow_builder.add_node("intake", intake_node)
workflow_builder.add_node("planner", planner_node)
workflow_builder.add_node("specialists", lambda state: {})
workflow_builder.add_node("present_plan", present_plan_node)
workflow_builder.add_node("travel_agent", travel_graph)
workflow_builder.add_node("accommodation_agent", hotel_graph)

workflow_builder.add_edge(START, "intake")
workflow_builder.add_edge("intake", "planner")
workflow_builder.add_conditional_edges(
    "planner",
    decide_next_step_after_planning,
    {"specialists": "specialists", "present_plan": "present_plan"}
)
workflow_builder.add_conditional_edges(
    "specialists",
    route_to_both_agents,
    ["travel_agent", "accommodation_agent"]
)
workflow_builder.add_edge(["travel_agent", "accommodation_agent"], "present_plan")
workflow_builder.add_edge("present_plan", END)

app = workflow_builder.compile(checkpointer=memory_saver)


# 10. Testing the Application

if __name__ == "__main__":
    
    origin = input("Origin City (e.g., Mumbai, New York, London) [default: New York]: ").strip() or "New York"
    destination = input("Destination City (e.g., Paris, Delhi, Tokyo) [default: Paris]: ").strip() or "Paris"
    currency = input("Currency (USD, EUR, INR, etc.) [default: USD]: ").strip().upper() or "USD"
    budget = input("Budget [default: 2000]: ").strip() or "2000"
    num_people = input("Number of Travelers [default: 1]: ").strip() or "1"
    arrival_date = input("Arrival Date (YYYY-MM-DD) [default: 2025-06-01]: ").strip() or "2025-06-01"
    return_date = input("Return Date (YYYY-MM-DD) [default: 2025-06-07]: ").strip() or "2025-06-07"
    
    arrival_date_object = datetime.strptime(return_date, "%Y-%m-%d")
    return_date_object = datetime.strptime(arrival_date, "%Y-%m-%d")
    stay_duration = str((arrival_date_object - return_date_object).days)
    
    user_inputs = {
        "messages": [],
        "origin": origin,
        "destination": destination,
        "currency": currency,
        "budget": budget,
        "num_people": num_people,
        "arrival_date": arrival_date,
        "return_date": return_date,
        "stay_duration": stay_duration,
        "flight_options": [],
        "hotel_options": [],
        "date_error": ""
    }
    
    unique_thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": unique_thread_id}}
    
    result = app.invoke(user_inputs, config)
    
    final_message = result['messages'][-1]
    print(final_message.content)


Planning trip: New York -> London
   Dates: 2025-12-19 to 2025-12-29
   Budget: 10000 USD

Travel Agent: Searching flights...

   [AMADEUS] Searching Flights: New York -> London on 2025-12-19

Hotel Agent: Searching hotels...

   [AMADEUS] Searching Hotels in: London
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Found 1 hotels
   Total hotels found: 1
   Found 3 flights

   [AMADEUS] Searching Flights: London -> New York on 2025-12-29
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Found 3 flights
   Total flights found: 6
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
   Converted 'New York' -> 'NYC'
   Converted 'London' -> 'LON'
# Your Trip Pla